4.1.2 파이토치로 MLP 구현하기

In [1]:
# 파이토치를 사용한 MLP
import torch.nn as nn
import torch.nn.functional as F

class MultilayerPerceptron(nn.Module):
    """
    """
    def __init__(self, input_size, hidden_size=2, output_size=3, 
                 num_hidden_layers=1, hidden_activation=nn.Sigmoid):
        """가중치 초기화

        매개변수:
            input_size (int): 입력 크기
            hidden_size (int): 은닉층 크기
            output_size (int): 출력 크기
            num_hidden_layers (int): 은닉층 개수
            hidden_activation (torch.nn.*): 활성화 함수
        """
        super(MultilayerPerceptron, self).__init__()
        self.module_list = nn.ModuleList()
        
        interim_input_size = input_size
        interim_output_size = hidden_size
        
        for _ in range(num_hidden_layers):
            self.module_list.append(nn.Linear(interim_input_size, interim_output_size))
            self.module_list.append(hidden_activation())
            interim_input_size = interim_output_size
            
        self.fc_final = nn.Linear(interim_input_size, output_size)
        
        self.last_forward_cache = []
       
    def forward(self, x, apply_softmax=False):
        """MLP의 정방향 계산
        
        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, input_dim)입니다.
            apply_softmax (bool): 소프트맥스 함수를 위한 플래그
                크로스 엔트로피 손실을 사용하려면 반드시 False로 지정해야 합니다
        반환값:
            결과 텐서. tensor.shape는 (batch, output_dim)입니다.
        """
        self.last_forward_cache = []
        self.last_forward_cache.append(x.to("cpu").numpy())

        for module in self.module_list:
            x = module(x)
            self.last_forward_cache.append(x.to("cpu").data.numpy())
            
        output = self.fc_final(x)
        self.last_forward_cache.append(output.to("cpu").data.numpy())

        if apply_softmax:
            output = F.softmax(output, dim=1)
            
        return output

In [2]:
# MLP 객체 생성
batch_size = 2 # 한 번에 입력할 샘플 개수
input_dim = 3
hidden_dim = 100
output_dim = 4

# 모델 생성
mlp = MultilayerPerceptron(input_dim, hidden_dim, output_dim)

print(mlp)

MultilayerPerceptron(
  (module_list): ModuleList(
    (0): Linear(in_features=3, out_features=100, bias=True)
    (1): Sigmoid()
  )
  (fc_final): Linear(in_features=100, out_features=4, bias=True)
)


In [4]:
# 랜덤한 입력으로 MLP 테스트하기
import torch

def describe(x):
    print("타입: {}".format(x.type()))
    print("크기: {}".format(x.shape))
    print("값: \n{}".format(x))

x_input = torch.rand(batch_size, input_dim)
describe(x_input)

print()

y_output = mlp(x_input, apply_softmax=False)
describe(y_output)

타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[0.3012, 0.5836, 0.4412],
        [0.9673, 0.3079, 0.2104]])

타입: torch.FloatTensor
크기: torch.Size([2, 4])
값: 
tensor([[-0.3718,  0.1920, -0.4468, -0.0963],
        [-0.4084,  0.2559, -0.4296, -0.0992]], grad_fn=<AddmmBackward>)


In [5]:
# MLP 분류기로 확률 출력하기(apply_softmax=True)
y_output = mlp(x_input, apply_softmax=True)
describe(y_output)

타입: torch.FloatTensor
크기: torch.Size([2, 4])
값: 
tensor([[0.1999, 0.3513, 0.1855, 0.2633],
        [0.1892, 0.3677, 0.1853, 0.2578]], grad_fn=<SoftmaxBackward>)


4.2.1 성씨 데이터셋

In [ ]:
# SurnameDataset.__getitem__() 구현
class SurnameDataset(Dataset):
    # [코드 3-14]와 구현이 매우 비슷합니다.

    def __getitem__(self, index):           
            row = self._target_df.iloc[index]

            surname_vector = self._vectorizer.vectorize(row.surname)

            nationality_index = self._vectorizer.nationality_vocab.lookup_token(row.nationality)

            return {'x_surname': surname_vector, 'y_nationality': nationality_index}